In [1]:
import torch
import torch.nn as nn
from torch import optim

import sys

In [2]:
# local imports 
sys.path.append("..")

from transformer_model import generate_text, train_batch, tokenize_raw_data
from data import ArticlesIter, articles_filepath

2023-03-30 20:26:32.449210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

## Setup

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load Vocab

In [ ]:
SAVE_PATH = "../transformer_model/vocab_save/" + "embeddings_vocab.pt"

emb_vocab_obj = torch.load(SAVE_PATH)

print("Vocab size:", len(emb_vocab_obj))

### Load Model

In [ ]:
SAVE_PATH = "../transformer_model/model_save/" + "glove_emb_model.pt"

emb_model = torch.load(SAVE_PATH)

print("Loaded hyperparameters:")
print("---------------------------")
print("Embedding Dimension:", emb_model.embedding_dim)
print("Hidden Dim:         ", emb_model.hidden_dim)
print("Num Layers:         ", emb_model.num_layers)
print("Num Heads:          ", emb_model.num_heads)
print("Max Length:         ", emb_model.max_len)
print("Dropout Per.:       ", emb_model.dropout_p)

## Test Text Generation
Given our mappings we just read in we can now have the transformer make some text.

In [ ]:
# start_str = "I went to the"
# number_of_words = 10

# print(start_str, end=" ")
        
# for word in generate_text(
#     model=emb_model,
#     vocab=emb_vocab_obj,
#     start_str=start_str,
#     max_length=number_of_words):
#     print(word, end=" ")

## Train the Model

In [ ]:
learning_rate = 3e-4

optimizer = optim.Adam(emb_model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

The data will be split into 1000 batches. For each batch, the content will be pulled out. This content will be tokenized and fed into the *train_model* function.

In [ ]:
article_iter = ArticlesIter(batch_size=1000)
total_loss = 0.0
total_batches = 0

for batch_i, batch in enumerate(article_iter):
    if batch_i < 5:
        print("Training batch:", batch_i)
        batch_content = batch["Content"]

        # for article in batch_content:
        #     tokenized_batch_content = tokenize_raw_data(article, emb_vocab_obj)

        #     print(article)
        #     print(tokenized_batch_content)

        total_loss += train_batch(
            model=emb_model,
            vocab=emb_vocab_obj,
            optimizer=optimizer,
            criterion=criterion,
            batch=batch_content,
            device=device
        )
        total_batches += 1

print("Average Loss:", total_loss/total_batches)
